In [1]:
from numcosmo_py import Ncm, Nc, GObject

import numpy as np
import math
import sys
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt

sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/")
sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CosmoDC2+RedMaPPer")

sys.path.insert(0, "/global/homes/c/cinlima/gcr-catalogs")
import GCRCatalogs

sys.path.insert(0, "global/homes/c/cinlima/clevar/clevar")

#Put nerc if using nersc
GCRCatalogs.set_root_dir_by_site("nersc")

# from clevar import ClCatalog, MemCatalog
# from clevar.match import MembershipMatch, get_matched_pairs
# from clevar.match import output_matched_catalog
# from clevar.match_metrics.scaling import ClCatalogFuncs as s_cf

# import clevar

from richness_mass_calib import create_richness_mass_calib

In [2]:
from getdist import plots
from getdist.mcsamples import  MCSamples
import getdist
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))

import pygtc
import pandas as pd
from IPython.display import display, Math


GetDist Version: 1.4.7, Matplotlib version: 3.9.1


In [3]:
# #Settings
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [4]:
# # Data
# DC2_halos_m200c = fits.open(
#     "/global/cfs/projectdirs/lsst/groups/CL/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits"
# )
# dt_halos = Table(DC2_halos_m200c[1].data)
# # dt_halos['redshift_true', 'mass_fof', 'm200c', 'richness']


In [5]:
# min_richness = 5
# min_halo_mass = 1e13 #Msun

# #Halos/Clusters
# #SkySim + RedMaPPer: 'skysim5000_v1.1.1_redmapper_v0.8.5'
# RM_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})

# #SkySim: 'skysim5000_v1.1.1_image'
# halo_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})

# #Members
# RM_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
# halo_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)

# #Match catalogs
# mt = MembershipMatch()

# mt.load_matches(RM_cat, halo_cat, out_dir='/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/SS_RM_match_cross_membership')

# RM_cat_bij, halo_cat_bij = get_matched_pairs(RM_cat, halo_cat, 'cross')

# clusters_dt = Table([halo_cat_bij['mass_fof'], RM_cat_bij['richness'], RM_cat_bij['z']], names=('m200c', 'richness', 'redshift_true'))

# dt_halos = clusters_dt

test_matchdata = pd.read_csv('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/MatchingCatalogs/match_catalog.csv',  sep = ';')

rich_data = test_matchdata["richness"]
z_data = test_matchdata["redshift_x"]
mass_data = test_matchdata["halo_mass"]

test_dt = Table([mass_data, rich_data, z_data], names=('m200c', 'richness', 'redshift_true'))
dt_halos = test_dt

In [10]:
ascaso = Nc.ClusterMassAscaso()
fixed_parameters_ascaso = [6] #fixing cut parameter

ascaso.param_set_by_name("cut", 1e15) #Set cut parameter value 
mset = Ncm.MSet()

lnrich_ext = Nc.ClusterMassLnrichExt()

def catalog_fit(dt_halos, model):
    rmdata = create_richness_mass_calib(dt_halos)
    dset = Ncm.Dataset.new()
    dset.append_data(rmdata)

    lh = Ncm.Likelihood.new(dset)
    
    if model:     
        mset.set(ascaso)
        rmdata.m2lnL_val(mset)  
        mset.param_set_all_ftype(Ncm.ParamType.FREE) #All parameters free

        #All parameters free except cut parameters:
        for par in fixed_parameters_ascaso:
            mset.param_set_ftype(7000, par, Ncm.ParamType.FIXED)

        mset.prepare_fparam_map()

    else:
        fixed_parameters_lnrich_ext = [ 13, 14, 15] #fixing cut parameters

        lnrich_ext.param_set_by_name("cut", 1e15) #Set cut parameter value 
        mset.set(lnrich_ext)
        rmdata.m2lnL_val(mset)  
        mset.param_set_all_ftype(Ncm.ParamType.FREE) #All parameters free

        #All parameters free except cut parameters:
        for par in fixed_parameters_lnrich_ext:
            mset.param_set_ftype(7000, par, Ncm.ParamType.FIXED)

        mset.prepare_fparam_map()

    fit = Ncm.Fit.factory( Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL )
    fit.log_info()
    fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)
    fit.log_info()
        
    return fit

fit = catalog_fit(dt_halos, model=True)

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  3.19                [FREE]
#   -    mup1[01]:  0.868588963806504   [FREE]
#   -    mup2[02]: -0.304006137332276   [FREE]
#   - sigmap0[03]:  0.33                [FREE]
#   - sigmap1[04]: -0.0347435585522601  [FREE]
#   - sigmap2[05]:  0                   [FREE]
#   -     cut[06]:  1e+15               [FIXED]
#----------------------------------------------------------------------------------
# Model fitting. Interating using:
#  - solver:            NLOpt:ln-neldermead
#  - differentiation:   Numerical differentiantion (central)
#........................................................

In [11]:
fit.log_info()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  3.03813575534708    [FREE]
#   -    mup1[01]:  0.358988294901382   [FREE]
#   -    mup2[02]:  0.0350584792566055  [FREE]
#   - sigmap0[03]:  0.642320017284801   [FREE]
#   - sigmap1[04]:  0.105692710740348   [FREE]
#   - sigmap2[05]:  0.107550790942883   [FREE]
#   -     cut[06]:  1e+15               [FIXED]


In [12]:
nwalkers = 1500

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("ascaso2.fits")

esmcmc.start_run()
esmcmc.run(1000)  
esmcmc.end_run()

esmcmc.mean_covar()
fit.log_covar()

# NcmThreadPool:Unused:      0
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      2
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensemble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 1500.
#   Number of threads: 0002.
#   Using MPI:         no - use MPI enabled but no slaves available.
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Model set:
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  3.03813575534708    [FREE]
#   -    mup1[01]: 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



# Task:NcmFitESMCMC, current time:        Wed Sep 11 2024, 05:06:37
# Task:NcmFitESMCMC, estimated to end at: Wed Sep 11 2024, 05:29:59 +/- 00:01:15.3621
#  Elapsed time: 00 days, 00:00:00.2906540
#  iteration            [000127]
#  function evaluations [000129]
#  gradient evaluations [000000]
#  degrees of freedom   [039594]
#  m2lnL     =     20847.9734394012 (     20841.847 )
#  Fit parameters:
#     3.03830286432191       0.359054843952539      0.0352538761084791     0.642286722881836      0.105661419420249      0.107248561879944     
# NcmMSetCatalog: Current mean:   20877        3.0382       0.35905      0.035293     0.64187      0.10549      0.10697    
# NcmMSetCatalog: Current msd:    0.49902      2.1241e-05   7.08e-06     2.4114e-05   9.8596e-06   3.7101e-06   1.89e-05   
# NcmMSetCatalog: Current sd:     360.54       0.015346     0.0051153    0.017422     0.0071235    0.0026805    0.013655   
# NcmMSetCatalog: Current var:    1.2999e+05   0.00023551   2.6166e-05   0.0003035

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



  1            1            1            1          
# NcmMSetCatalog: Current skfac:  1            1.0208       1.0245       1.0161       1.019        1.0184       1.019      
# NcmMSetCatalog: Maximal Shrink factor = 1.02487               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `375':
# NcmMSetCatalog: Constant break point at `103':
# NcmFitESMCMC:acceptance ratio 38.2064% (last update 47.5333%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCMC:last ensemble variance of -2ln(L):       11.7910166308757 (2n = 12), min(-2ln(L)) =        20841.898127134.
# Task:NcmFitESMCMC, completed: 562500 of 1500000, elapsed time: 00:13:25.1861
# Task:NcmFitESMCMC, mean time: 00:00:00.0014 +/- 00:00:00.0001
# Task:NcmFitESMCMC, time left: 00:22:21.9194 +/- 00:01:09.4078
# Task:NcmFitESMCMC, current time:        Wed Sep 11 2024, 05:07:37
# Task:NcmFitESMCMC, es

...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



#  Elapsed time: 00 days, 00:00:00.2906540
#  iteration            [000127]
#  function evaluations [000129]
#  gradient evaluations [000000]
#  degrees of freedom   [039594]
#  m2lnL     =     20847.7847257256 (     20841.847 )
#  Fit parameters:
#     3.03813754971505       0.359001225543184      0.0353803180574064     0.642189545877653      0.105637408376028      0.107821965802937     
# NcmMSetCatalog: Current mean:   20866        3.0382       0.35903      0.035208     0.64204      0.10556      0.10717    
# NcmMSetCatalog: Current msd:    0.32196      1.4587e-05   4.9342e-06   1.7557e-05   6.9943e-06   2.6096e-06   1.3481e-05 
# NcmMSetCatalog: Current sd:     289.76       0.013128     0.0044408    0.015801     0.0062949    0.0023487    0.012133   
# NcmMSetCatalog: Current var:    83963        0.00017234   1.9721e-05   0.00024967   3.9625e-05   5.5162e-06   0.00014721 
# NcmMSetCatalog: Current tau:    1            1            1            1            1            1            

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



#  Elapsed time: 00 days, 00:00:00.2906540
#  iteration            [000127]
#  function evaluations [000129]
#  gradient evaluations [000000]
#  degrees of freedom   [039594]
#  m2lnL     =     20847.8902645021 (     20841.847 )
#  Fit parameters:
#     3.03815528066658       0.35897951812041       0.0354207952119914     0.642384865788354      0.105719757015249      0.107724477851053     
# NcmMSetCatalog: Current mean:   20859        3.0382       0.35902      0.035167     0.64216      0.10561      0.1073     
# NcmMSetCatalog: Current msd:    0.19101      9.5916e-06   3.3086e-06   1.2399e-05   4.7866e-06   1.7673e-06   9.3117e-06 
# NcmMSetCatalog: Current sd:     223.29       0.011212     0.0038677    0.014494     0.0055954    0.002066     0.010885   
# NcmMSetCatalog: Current var:    49857        0.00012572   1.4959e-05   0.00021009   3.1309e-05   4.2682e-06   0.00011849 
# NcmMSetCatalog: Current tau:    1            1            1            1            1            1            

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [13]:
esmcmc.mean_covar()
fit.log_covar()

#----------------------------------------------------------------------------------
# NcmMSet parameters covariance matrix
#                                                   -------------------------------------------------------------------------------------------
#    mup0[07000:00] =  3.038       +/-  0.01094     |  1           |  0.8976      | -0.0395      | -0.3219      | -0.3633      | -0.05427     |
#    mup1[07000:01] =  0.359       +/-  0.003787    |  0.8976      |  1           | -0.01905     | -0.3712      | -0.4215      | -0.05519     |
#    mup2[07000:02] =  0.03517     +/-  0.01432     | -0.0395      | -0.01905     |  1           | -0.03044     | -0.01298     |  0.09409     |
# sigmap0[07000:03] =  0.6422      +/-  0.005498    | -0.3219      | -0.3712      | -0.03044     |  1           |  0.9153      |  0.03233     |
# sigmap1[07000:04] =  0.1056      +/-  0.002026    | -0.3633      | -0.4215      | -0.01298     |  0.9153      |  1           |  0.07503     |
# sigmap2[070

In [14]:
fit.log_info()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  3.03816669498779    [FREE]
#   -    mup1[01]:  0.359014231147651   [FREE]
#   -    mup2[02]:  0.0351651782238234  [FREE]
#   - sigmap0[03]:  0.642182904026122   [FREE]
#   - sigmap1[04]:  0.105618715078116   [FREE]
#   - sigmap2[05]:  0.107324825119421   [FREE]
#   -     cut[06]:  1e+15               [FIXED]


In [27]:
data_fit_full = pd.DataFrame(fits.open("ascaso2.fits")[1].data).iloc[:, 1:7].T
data_fit_void = np.array(data_fit_full)
data_fit = []
for item in data_fit_void:
    arr= np.array(item)
    data_fit.append(np.asarray(arr.tolist()))

names = [
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
]
labels=["\mu_{0}", "\mu_{1}", "\mu_{2}", "\sigma_{0}", "\sigma_{1}", "\sigma_{2}"]
settings = {
    "mult_bias_correction_order": 0,
    "smooth_scale_2D": 3,
    "smooth_scale_1D": 3,
    "boundary_correction_order": 0,
}
samples1 = MCSamples(samples=data_fit, names=names, labels=labels, settings=settings)
samples1.removeBurn(0.3)

Removed no burn in


In [30]:
# Triangle plot
g1 = plots.get_subplot_plotter()

g1.settings.legend_fontsize = 18
g1.settings.lab_fontsize = 25
g1.settings.axes_fontsize = 20


g1.triangle_plot(
    [samples1],
    filled=True,
    contour_ls="-",
    contour_lws=1,
    fontsize=14,
    fine_bins=1,
    colors=['#a0d995'],
    line_args=[{'lw':1.2,'ls':'-', 'color':'#a0d995'}],
    legend_labels=["Ascaso"],
)

plt.show()
for i in range(1, 7):
    display(Math(samples1.getInlineLatex(str(i),limit=1)))
plt.savefig('mcmc3.png')



<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
data_fit_full = pd.DataFrame(fits.open("ascaso.fits")[1].data).iloc[:, 1:14].T
data_fit_void = np.array(data_fit_full)
data_fit = []
for item in data_fit_void:
    arr= np.array(item)
    data_fit.append(np.asarray(arr.tolist()))

names = [
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
    '7',
    '8',
    '9',
    '10',
    '11',
    '12',
    '13'
]
labels=["\mu_0", "\mu_{M1}", "\mu_{M2}", "\mu_{Z1}"," \mu_{Z2}", "\mu_{MZ}", "\sigma_0", "\sigma_{M1}", "\sigma_{M2}", "\sigma_{Z1}", "\sigma_{Z2}", "\sigma_{MZ}","A_0" ]
settings = {
    "mult_bias_correction_order": 0,
    "smooth_scale_2D": 3,
    "smooth_scale_1D": 3,
    "boundary_correction_order": 0,
}
samples2 = MCSamples(samples=data_fit, names=names, labels=labels, settings=settings)
samples2.removeBurn(0.3)


In [ ]:
# Triangle plot
g2 = plots.get_subplot_plotter()

g2.settings.lab_fontsize = 40
g2.settings.legend_fontsize = 40
g2.settings.axes_fontsize = 30

g2.triangle_plot(
    [samples2],
    filled=True,
    contour_ls="-",
    contour_lws=1,
    fine_bins=1,
    colors=['#b186f1'],
    fontsize=70,
    line_args=[{'lw':1.2,'ls':'-', 'color':'#b186f1'}],
    legend_labels=["Estendido", ],
)
#plt.savefig('mcmc.png')
plt.show()


In [ ]:
for i in range(1, 14):
    display(Math(samples2.getInlineLatex(str(i),limit=1)))


In [ ]:
min_richness = 5
min_halo_mass = 1e13 #Msun

#Halos/Clusters
#CosmoDC2 + RedMaPPer: 'cosmoDC2_v1.1.4_redmapper_v0.8.1'
RM_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})

#CosmoDC2: 'cosmoDC2_v1.1.4_image'
halo_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})

#Members
RM_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
halo_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)

#Match catalogs
mt = MembershipMatch()

mt.load_matches(RM_cat, halo_cat, out_dir='/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/cosmoDC2_RM_match_cross_membership')

#Matched Data
RM_cat_bij, halo_cat_bij = get_matched_pairs(RM_cat, halo_cat, 'cross')

cdc2_dt = Table([halo_cat_bij['mass_fof'], RM_cat_bij['richness'], RM_cat_bij['z']], names=('m200c', 'richness', 'redshift_true'))
dt_halos = cdc2_dt


fit = catalog_fit(dt_halos, ascaso=False)


In [ ]:
nwalkers = 1500

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("mcmc2.fits")

esmcmc.start_run()
esmcmc.run(1000)  
esmcmc.end_run()

esmcmc.mean_covar()


In [ ]:
fit.log_covar()

In [ ]:
data_fit_full = pd.DataFrame(fits.open("mcmc2.fits")[1].data).iloc[:, 1:14].T
data_fit_void = np.array(data_fit_full)
data_fit = []
for item in data_fit_void:
    arr= np.array(item)
    data_fit.append(np.asarray(arr.tolist()))

names = [
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
    '7',
    '8',
    '9',
    '10',
    '11',
    '12',
    '13'
]
labels=["\mu_0", "\mu_{M1}", "\mu_{M2}", "\mu_{Z1}"," \mu_{Z2}", "\mu_{MZ}", "\sigma_0", "\sigma_{M1}", "\sigma_{M2}", "\sigma_{Z1}", "\sigma_{Z2}", "\sigma_{MZ}","A_0" ]
settings = {
    "mult_bias_correction_order": 0,
    "smooth_scale_2D": 3,
    "smooth_scale_1D": 3,
    "boundary_correction_order": 0,
}
samples3 = MCSamples(samples=data_fit, names=names, labels=labels, settings=settings)
samples3.removeBurn(0.3)

In [ ]:
# Triangle plot
g2 = plots.get_subplot_plotter()

g2.settings.lab_fontsize = 40
g2.settings.legend_fontsize = 40
g2.settings.axes_fontsize = 30

g2.triangle_plot(
    [samples3],
    filled=True,
    contour_ls="-",
    contour_lws=1,
    fine_bins=1,
    colors=['#b186f1'],
    fontsize=70,
    line_args=[{'lw':1.2,'ls':'-', 'color':'#b186f1'}],
    legend_labels=["Estendido", ],
)
plt.savefig('mcmc2.png')
plt.show()


In [ ]:
for i in range(1, 14):
    display(Math(samples3.getInlineLatex(str(i),limit=1)))
